<img src="https://docs.actable.ai/_images/logo.png" style="object-fit: cover; max-width:100%; height:300px;" />

# infer_causal

This notebook is an example on how you can run a causal inference analyisis automatically with
[Actable AI](https://actable.ai)

For this example we will see the average effect of a treatment on an outcome\
and then see how controlling by a certain feature gives us multiple average effects.

### Imports

This part simply imports the python modules.
Last line imports the infer_causal from actableai

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from actableai import infer_causal
from actableai.utils.testing import unittest_hyperparameters

### Importing the data

This part simply imports the data.

In [ ]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/Actable-AI/public-datasets/master/apartments.csv"
).head(100)
df

### Calling Actable AI task

This part is the call to the ActableAI causal inference analysis.\
To learn more about the available parameters you can consult the [API Documentation](https://lib.actable.ai/actableai.tasks.html#actableai.tasks.causal_inference.infer_causal)

In [ ]:
# Here df is the DataFrame containing our data
# target is "Churn" because we want to predict the churn
# features set to None means that we will use every single feature available
result = infer_causal(pd_table=df, treatments=["location"], outcomes=["rental_price"])


### Evaluation of the generated model

In [ ]:
result